In [ ]:
# public libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
sc.settings.set_figure_params(dpi=100)

In [ ]:
import tensorflow as tf
from tensorflow import keras

%load_ext tensorboard

In [ ]:
from constants import *
import multiprocessing as mp
from tqdm import tqdm

## DEFINE CONSTANTS
stretch_length_cdr3 = 30
stretch_length_ag = 15

## DEFINE ENCODERS
def _encode_cdr3s(peptide):
    # define columns
    columns = cdr3_alphabet+['charge','hydrophobicity','weight','sulfur','aromatic']
    columns_returned = [f'{idx}{col}' for idx in range(stretch_length_cdr3) for col in columns]
    # create the tracking dataframe
    X = pd.DataFrame(np.nan, index=range(stretch_length_cdr3), columns=columns)
    step = stretch_length_cdr3 / len(peptide)
    for idx, aa in enumerate(peptide):
        # find start and end of each peptide
        start, end = round(idx * step), round((idx + 1) * step)
        if idx == len(peptide) - 1:
            end = stretch_length_cdr3
        # map to the coordinates
        charge = amino_acid_charge[aa]
        hydrophobicity = amino_acid_hydrophobicity[aa] / 2
        weight = amino_acid_weights[aa] / stretch_length_cdr3
        sulfur = 1 * (aa in has_sulfur)
        aromatic = 1 * (aa in is_aromatic)
        X.loc[start:end, cdr3_alphabet] = 0
        X.loc[start:end, aa] = 1
        X.loc[start:end, ['charge','hydrophobicity','weight','sulfur','aromatic']] = charge, hydrophobicity, weight, sulfur, aromatic
    assert not X.isna().any().any()
    return pd.Series(X.values.flatten(), index=columns_returned, name=peptide)
def encode_cdr3s(cdr3s, n_cpus, verbose):
    if verbose: print('\n\tencoding TCRs...', end='')
    # the first step is to create a conversion map for CDR3 sequences
    global cdr3_alphabet
    cdr3_alphabet = sorted(set([el for x in cdr3s for el in list(x)]))
    # work through cdr3s
    els = []
    with mp.Pool(n_cpus) as pool:
        for el in pool.imap_unordered(_encode_cdr3s, cdr3s):
            els.append(el)
    cdr3_to_X = pd.concat(els, axis=1).T
    if verbose: print('done!')
    return cdr3_to_X


def _encode_ags(peptide):
    # define columns
    columns = ag_alphabet+['charge','hydrophobicity','weight','sulfur','aromatic']
    columns_returned = [f'{idx}{col}' for idx in range(stretch_length_ag) for col in columns]
    # create the tracking dataframe
    X = pd.DataFrame(np.nan, index=range(stretch_length_ag), columns=columns)
    step = stretch_length_ag / len(peptide)
    for idx, aa in enumerate(peptide):
        # find start and end of each peptide
        start, end = round(idx * step), round((idx + 1) * step)
        if idx == len(peptide) - 1:
            end = stretch_length_ag
        # map to the coordinates
        charge = amino_acid_charge[aa]
        hydrophobicity = amino_acid_hydrophobicity[aa] / 2
        weight = amino_acid_weights[aa] / stretch_length_ag
        sulfur = 1 * (aa in has_sulfur)
        aromatic = 1 * (aa in is_aromatic)
        X.loc[start:end, ag_alphabet] = 0
        X.loc[start:end, aa] = 1
        X.loc[start:end, ['charge','hydrophobicity','weight','sulfur','aromatic']] = charge, hydrophobicity, weight, sulfur, aromatic
    assert not X.isna().any().any()
    return pd.Series(X.values.flatten(), index=columns_returned, name=peptide)
def encode_ags(ags, n_cpus, verbose):
    if verbose: print('\tencoding antigens...', end='')
    # the first step is to create a conversion map for ag sequences
    global ag_alphabet
    ag_alphabet = sorted(set([el for x in ags for el in list(x)]))
    # work through antigens
    els = []
    with mp.Pool(n_cpus) as pool:
        for el in pool.imap_unordered(_encode_ags, ags):
            els.append(el)
    ag_to_X = pd.concat(els, axis=1).T
    if verbose: print('done!')
    return ag_to_X

In [ ]:
# we'll read in the train and test
train = pd.read_csv('../external/tcr-specificity-prediction-challenge/VDJdb_paired_chain.csv')
test = pd.read_csv('../external/tcr-specificity-prediction-challenge/test.csv')
# retrieve encodings
cdr3_to_X = encode_cdr3s(pd.concat([train, test], axis=0)['CDR3b_extended'].unique(), 40, False)
ag_to_X = encode_cdr3s(pd.concat([train, test], axis=0)['Peptide'].unique(), 40, False)
# get the formattings to match
train['HLA'] = train['HLA'].str.replace('HLA-','')

In [ ]:
# save the originals
train_orig, test_orig = train.copy(), test.copy()

## tcrb only v1

In [ ]:
# read in the levenshtein distance for the epitope information
from Levenshtein import distance
from tqdm import tqdm
# our goal here is to find the similar and different peptides in our dataset
counts = train['Peptide'].value_counts()
train_l = pd.DataFrame(index=counts.index, columns=counts.index)
for idx, pep1 in tqdm(enumerate(counts.index[:-1]), total=train_l.shape[0]):
    train_l.loc[pep1, pep1] = 0
    for pep2 in counts.index[idx+1:]:
        d = distance(pep1, pep2)
        train_l.loc[pep1, pep2] = d
        train_l.loc[pep2, pep1] = d
train_l.loc[pep2, pep2] = 0

In [ ]:
# retrieve hla specific predictions
hla2pred = {}
for hla in test_orig['HLA'].unique():
    # subset for HLA
    train = train_orig.loc[train_orig['HLA'] == hla].copy()
    test = test_orig.loc[test_orig['HLA'] == hla].copy()

    from tqdm import tqdm
    # set seed and identify irrelevant matches
    np.random.seed(0)
    irrs = []

    # re count the peptides for this HLA
    counts = train['Peptide'].value_counts()
    for pep in tqdm(counts.index):
        # gather peptide cdr3 information
        n_cdr3s = counts.loc[pep]
        pep_cdr3s = train.loc[train['Peptide'] == pep, 'CDR3b_extended'].unique()
        # we first grab the peptide levenshtein distances
        pep_levenshtein = train_l[pep].sort_values()[::-1]
        # systematically identify negative controls
        irr_cdr3s = []
        while n_cdr3s > 0:
            # then we look at the max distance, gather those peptides, randomly choose one
            # find associated CDR3s that don't overlap with the current peptide
            vmax = pep_levenshtein.max()
            irr_peps = pep_levenshtein.index[pep_levenshtein == vmax]
            # reset for the next round if needed
            pep_levenshtein = pep_levenshtein[pep_levenshtein < vmax]
            # find the irrelevant peptide CDR3s and make sure they don't overlap
            mask = (train_orig['Peptide'].isin(irr_peps)) & (~train_orig['CDR3b_extended'].isin(pep_cdr3s))
            if sum(mask) == 0:
                continue
            # if we have cdr3s then grab them out
            cdr3s = train_orig.loc[mask, 'CDR3b_extended']
            # if there are more than we need randomly select
            if len(cdr3s) > n_cdr3s:
                irr_cdr3s += np.random.choice(cdr3s, size=n_cdr3s, replace=False).tolist()
                break
            # otherwise keep moving
            else:
                irr_cdr3s += cdr3s.tolist()
                n_cdr3s -= len(cdr3s)
        # compile the full list
        irr = pd.DataFrame(irr_cdr3s, columns=['CDR3b_extended'])
        irr['Peptide'] = pep
        irrs.append(irr)

    from sklearn.metrics import roc_curve, auc, accuracy_score

    ## CONVERT TO CORRECT FORMAT
    # create X for training
    X_train = pd.concat([pd.concat(irrs, axis=0), train], axis=0)
    X_train_cdr3s = cdr3_to_X.loc[X_train['CDR3b_extended']].reset_index().iloc[:, 1:]
    X_train_epitopes = ag_to_X.loc[X_train['Peptide']].reset_index().iloc[:, 1:]
    # combine
    X_train_cdr3s.columns = 'cdr3:' + X_train_cdr3s.columns
    X_train_epitopes.columns = 'ag:' + X_train_epitopes.columns
    X_train = X_train_cdr3s.join(X_train_epitopes)

    # grab y for training
    y_train = pd.Series(0, index=X_train.index)
    y_train.iloc[-train.shape[0]:] = 1

    # confirm the same length
    assert X_train.shape[0] == y_train.shape[0]

    # create X for testing
    X_test_cdr3s = cdr3_to_X.loc[test['CDR3b_extended']].reset_index().iloc[:, 1:]
    X_test_epitopes = ag_to_X.loc[test['Peptide']].reset_index().iloc[:, 1:]
    # combine
    X_test_cdr3s.columns = 'cdr3:' + X_test_cdr3s.columns
    X_test_epitopes.columns = 'ag:' + X_test_epitopes.columns
    X_test = X_test_cdr3s.join(X_test_epitopes)

    # remove constant columns
    X_train = X_train.loc[:, X_train.nunique(0) > 1]
    X_train = X_train.loc[:, X_train.sum(0) > 0]
    # read in normalization factors
    means = X_train.mean(0)
    stds = (X_train - means).std(0)
    # subset for relevant columns
    X_train = X_train[means.index]
    X_test = X_test[means.index]
    # normalize
    X_train -= means
    X_train /= stds
    X_test -= means
    X_test /= stds


    ## SETUP MODEL
    # retrieve the appropriate columns
    cols_cdr3 = X_train.columns[X_train.columns.str.startswith('cdr3')]
    cols_ag = X_train.columns[X_train.columns.str.startswith('ag')]

    # determine model parameters
    # > layer for cdr3 alone
    input_1 = keras.layers.Input(shape=(len(cols_cdr3)))
    output_1 = keras.layers.Dense(200, activation='sigmoid')(input_1)
    # > layer for ag alone
    input_2 = keras.layers.Input(shape=(len(cols_ag)))
    output_2 = keras.layers.Dense(100, activation='sigmoid')(input_2)
    # > combined layer
    concat_3 = keras.layers.Concatenate()([output_1, output_2])
    output_3 = keras.layers.Dense(100, activation='sigmoid')(concat_3)
    # > final logit softmax layer
    output_4 = keras.layers.Dense(1, activation='sigmoid')(output_3)
    model = keras.Model(inputs=[input_1, input_2], outputs=[output_4])
    # set up the training parameters for the model
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss='binary_crossentropy',
        metrics=['accuracy','AUC'],
    )
    # train the model
    history = model.fit([X_train[cols_cdr3], X_train[cols_ag]], y_train,
                        epochs=10,
                        validation_data=([X_train[cols_cdr3], X_train[cols_ag]], y_train),
                        workers=40, use_multiprocessing=True)

    # testing predictions
    y_pred = model.predict([X_test[cols_cdr3], X_test[cols_ag]],
                           workers=40, use_multiprocessing=True)[:, 0]
    y_pred = pd.Series(y_pred, index=test.index, name='Pred%')
    hla2pred[hla] = y_pred

In [ ]:
# add back in predictions
test = test_orig.copy()
test['Prediction'] = np.nan
for pred in hla2pred.values():
    test.loc[pred.index, 'Prediction'] = pred
# write down predictions
test.to_csv('../external/tcr-specificity-prediction-challenge/submission.csv', index=False)

## tcrb only v2

In [ ]:
# retrieve hla specific predictions
hla2pred = {}
for hla in test_orig['HLA'].unique():
    # subset for HLA
    train = train_orig.loc[train_orig['HLA'] == hla].copy()
    test = test_orig.loc[test_orig['HLA'] == hla].copy()

    from tqdm import tqdm
    # set seed and identify irrelevant matches
    np.random.seed(0)
    irrs = []

    # re count the peptides for this HLA
    counts = train['Peptide'].value_counts()
    for pep in tqdm(counts.index):
        # gather peptide cdr3 information
        n_cdr3s = counts.loc[pep]
        pep_cdr3s = train.loc[train['Peptide'] == pep, 'CDR3b_extended'].unique()
        # we first grab the peptide levenshtein distances
        pep_levenshtein = train_l[pep].sort_values()[::-1]
        # systematically identify negative controls
        irr_cdr3s = []
        while n_cdr3s > 0:
            # then we look at the max distance, gather those peptides, randomly choose one
            # find associated CDR3s that don't overlap with the current peptide
            vmax = pep_levenshtein.max()
            irr_peps = pep_levenshtein.index[pep_levenshtein == vmax]
            # reset for the next round if needed
            pep_levenshtein = pep_levenshtein[pep_levenshtein < vmax]
            # find the irrelevant peptide CDR3s and make sure they don't overlap
            mask = (train_orig['Peptide'].isin(irr_peps)) & (~train_orig['CDR3b_extended'].isin(pep_cdr3s))
            if sum(mask) == 0:
                continue
            # if we have cdr3s then grab them out
            cdr3s = train_orig.loc[mask, 'CDR3b_extended']
            # if there are more than we need randomly select
            if len(cdr3s) > n_cdr3s:
                irr_cdr3s += np.random.choice(cdr3s, size=n_cdr3s, replace=False).tolist()
                break
            # otherwise keep moving
            else:
                irr_cdr3s += cdr3s.tolist()
                n_cdr3s -= len(cdr3s)
        # compile the full list
        irr = pd.DataFrame(irr_cdr3s, columns=['CDR3b_extended'])
        irr['Peptide'] = pep
        irrs.append(irr)

    from sklearn.metrics import roc_curve, auc, accuracy_score

    ## CONVERT TO CORRECT FORMAT
    # create X for training
    X_train = pd.concat([pd.concat(irrs, axis=0), train], axis=0)
    X_train_cdr3s = cdr3_to_X.loc[X_train['CDR3b_extended']].reset_index().iloc[:, 1:]
    X_train_epitopes = ag_to_X.loc[X_train['Peptide']].reset_index().iloc[:, 1:]
    # combine
    X_train_cdr3s.columns = 'cdr3:' + X_train_cdr3s.columns
    X_train_epitopes.columns = 'ag:' + X_train_epitopes.columns
    X_train = X_train_cdr3s.join(X_train_epitopes)

    # grab y for training
    y_train = pd.Series(0, index=X_train.index)
    y_train.iloc[-train.shape[0]:] = 1

    # confirm the same length
    assert X_train.shape[0] == y_train.shape[0]

    # create X for testing
    X_test_cdr3s = cdr3_to_X.loc[test['CDR3b_extended']].reset_index().iloc[:, 1:]
    X_test_epitopes = ag_to_X.loc[test['Peptide']].reset_index().iloc[:, 1:]
    # combine
    X_test_cdr3s.columns = 'cdr3:' + X_test_cdr3s.columns
    X_test_epitopes.columns = 'ag:' + X_test_epitopes.columns
    X_test = X_test_cdr3s.join(X_test_epitopes)

    # remove constant columns
    X_train = X_train.loc[:, X_train.nunique(0) > 1]
    X_train = X_train.loc[:, X_train.sum(0) > 0]
    # read in normalization factors
    means = X_train.mean(0)
    stds = (X_train - means).std(0)
    # subset for relevant columns
    X_train = X_train[means.index]
    X_test = X_test[means.index]
    # normalize
    X_train -= means
    X_train /= stds
    X_test -= means
    X_test /= stds


    ## SETUP MODEL
    # retrieve the appropriate columns
    cols_cdr3 = X_train.columns[X_train.columns.str.startswith('cdr3')]
    cols_ag = X_train.columns[X_train.columns.str.startswith('ag')]

    # determine model parameters
    # > layer for cdr3 alone
    input_1 = keras.layers.Input(shape=(len(cols_cdr3)))
    output_1 = keras.layers.Dense(300, activation='sigmoid')(input_1)
    # > layer for ag alone
    input_2 = keras.layers.Input(shape=(len(cols_ag)))
    output_2 = keras.layers.Dense(200, activation='sigmoid')(input_2)
    # > combined layer
    concat_3 = keras.layers.Concatenate()([output_1, output_2])
    output_3 = keras.layers.Dense(200, activation='sigmoid')(concat_3)
    # > final logit softmax layer
    output_4 = keras.layers.Dense(1, activation='sigmoid')(output_3)
    model = keras.Model(inputs=[input_1, input_2], outputs=[output_4])
    # set up the training parameters for the model
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss='binary_crossentropy',
        metrics=['accuracy','AUC'],
    )
    # train the model
    history = model.fit([X_train[cols_cdr3], X_train[cols_ag]], y_train,
                        epochs=10,
                        validation_data=([X_train[cols_cdr3], X_train[cols_ag]], y_train),
                        workers=40, use_multiprocessing=True)

    # testing predictions
    y_pred = model.predict([X_test[cols_cdr3], X_test[cols_ag]],
                           workers=40, use_multiprocessing=True)[:, 0]
    y_pred = pd.Series(y_pred, index=test.index, name='Pred%')
    hla2pred[hla] = y_pred

In [ ]:
# add back in predictions
test = test_orig.copy()
test['Prediction'] = np.nan
for pred in hla2pred.values():
    test.loc[pred.index, 'Prediction'] = pred
# write down predictions
test.to_csv('../external/tcr-specificity-prediction-challenge/submission.csv', index=False)

## tcrb only v3

In [ ]:
# retrieve hla specific predictions
hla2pred = {}
for hla in test_orig['HLA'].unique():
    # subset for HLA
    train = train_orig.loc[train_orig['HLA'] == hla].copy()
    test = test_orig.loc[test_orig['HLA'] == hla].copy()

    from tqdm import tqdm
    # set seed and identify irrelevant matches
    np.random.seed(0)
    irrs = []

    # re count the peptides for this HLA
    counts = train['Peptide'].value_counts()
    for pep in tqdm(counts.index):
        # gather peptide cdr3 information
        n_cdr3s = counts.loc[pep]
        pep_cdr3s = train.loc[train['Peptide'] == pep, 'CDR3b_extended'].unique()
        # we first grab the peptide levenshtein distances
        pep_levenshtein = train_l[pep].sort_values()[::-1]
        # systematically identify negative controls
        irr_cdr3s = []
        while n_cdr3s > 0:
            # then we look at the max distance, gather those peptides, randomly choose one
            # find associated CDR3s that don't overlap with the current peptide
            vmax = pep_levenshtein.max()
            irr_peps = pep_levenshtein.index[pep_levenshtein == vmax]
            # reset for the next round if needed
            pep_levenshtein = pep_levenshtein[pep_levenshtein < vmax]
            # find the irrelevant peptide CDR3s and make sure they don't overlap
            mask = (train_orig['Peptide'].isin(irr_peps)) & (~train_orig['CDR3b_extended'].isin(pep_cdr3s))
            if sum(mask) == 0:
                continue
            # if we have cdr3s then grab them out
            cdr3s = train_orig.loc[mask, 'CDR3b_extended']
            # if there are more than we need randomly select
            if len(cdr3s) > n_cdr3s:
                irr_cdr3s += np.random.choice(cdr3s, size=n_cdr3s, replace=False).tolist()
                break
            # otherwise keep moving
            else:
                irr_cdr3s += cdr3s.tolist()
                n_cdr3s -= len(cdr3s)
        # compile the full list
        irr = pd.DataFrame(irr_cdr3s, columns=['CDR3b_extended'])
        irr['Peptide'] = pep
        irrs.append(irr)

    from sklearn.metrics import roc_curve, auc, accuracy_score

    ## CONVERT TO CORRECT FORMAT
    # create X for training
    X_train = pd.concat([pd.concat(irrs, axis=0), train], axis=0)
    X_train_cdr3s = cdr3_to_X.loc[X_train['CDR3b_extended']].reset_index().iloc[:, 1:]
    X_train_epitopes = ag_to_X.loc[X_train['Peptide']].reset_index().iloc[:, 1:]
    # combine
    X_train_cdr3s.columns = 'cdr3:' + X_train_cdr3s.columns
    X_train_epitopes.columns = 'ag:' + X_train_epitopes.columns
    X_train = X_train_cdr3s.join(X_train_epitopes)

    # grab y for training
    y_train = pd.Series(0, index=X_train.index)
    y_train.iloc[-train.shape[0]:] = 1

    # confirm the same length
    assert X_train.shape[0] == y_train.shape[0]

    # create X for testing
    X_test_cdr3s = cdr3_to_X.loc[test['CDR3b_extended']].reset_index().iloc[:, 1:]
    X_test_epitopes = ag_to_X.loc[test['Peptide']].reset_index().iloc[:, 1:]
    # combine
    X_test_cdr3s.columns = 'cdr3:' + X_test_cdr3s.columns
    X_test_epitopes.columns = 'ag:' + X_test_epitopes.columns
    X_test = X_test_cdr3s.join(X_test_epitopes)

    # remove constant columns
    X_train = X_train.loc[:, X_train.nunique(0) > 1]
    X_train = X_train.loc[:, X_train.sum(0) > 0]
    # read in normalization factors
    means = X_train.mean(0)
    stds = (X_train - means).std(0)
    # subset for relevant columns
    X_train = X_train[means.index]
    X_test = X_test[means.index]
    # normalize
    X_train -= means
    X_train /= stds
    X_test -= means
    X_test /= stds


    ## SETUP MODEL
    # retrieve the appropriate columns
    cols_cdr3 = X_train.columns[X_train.columns.str.startswith('cdr3')]
    cols_ag = X_train.columns[X_train.columns.str.startswith('ag')]

    # determine model parameters
    # > layer for cdr3 alone
    input_1 = keras.layers.Input(shape=(len(cols_cdr3)))
    output_1 = keras.layers.Dense(400, activation='sigmoid')(input_1)
    # > layer for ag alone
    input_2 = keras.layers.Input(shape=(len(cols_ag)))
    output_2 = keras.layers.Dense(300, activation='sigmoid')(input_2)
    # > combined layer
    concat_3 = keras.layers.Concatenate()([output_1, output_2])
    output_3 = keras.layers.Dense(300, activation='sigmoid')(concat_3)
    # > final logit softmax layer
    output_4 = keras.layers.Dense(1, activation='sigmoid')(output_3)
    model = keras.Model(inputs=[input_1, input_2], outputs=[output_4])
    # set up the training parameters for the model
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss='binary_crossentropy',
        metrics=['accuracy','AUC'],
    )
    # train the model
    history = model.fit([X_train[cols_cdr3], X_train[cols_ag]], y_train,
                        epochs=10,
                        validation_data=([X_train[cols_cdr3], X_train[cols_ag]], y_train),
                        workers=40, use_multiprocessing=True)

    # testing predictions
    y_pred = model.predict([X_test[cols_cdr3], X_test[cols_ag]],
                           workers=40, use_multiprocessing=True)[:, 0]
    y_pred = pd.Series(y_pred, index=test.index, name='Pred%')
    hla2pred[hla] = y_pred

In [ ]:
# add back in predictions
test = test_orig.copy()
test['Prediction'] = np.nan
for pred in hla2pred.values():
    test.loc[pred.index, 'Prediction'] = pred
# write down predictions
test.to_csv('../external/tcr-specificity-prediction-challenge/submission.csv', index=False)

## tcrb only v4

In [ ]:
# retrieve hla specific predictions
hla2pred = {}
for hla in test_orig['HLA'].unique():
    # subset for HLA
    train = train_orig.loc[train_orig['HLA'] == hla].copy()
    test = test_orig.loc[test_orig['HLA'] == hla].copy()

    from tqdm import tqdm
    # set seed and identify irrelevant matches
    np.random.seed(0)
    irrs = []

    # re count the peptides for this HLA
    counts = train['Peptide'].value_counts()
    for pep in tqdm(counts.index):
        # gather peptide cdr3 information
        n_cdr3s = counts.loc[pep]
        pep_cdr3s = train.loc[train['Peptide'] == pep, 'CDR3b_extended'].unique()
        # we first grab the peptide levenshtein distances
        pep_levenshtein = train_l[pep].sort_values()[::-1]
        # systematically identify negative controls
        irr_cdr3s = []
        while n_cdr3s > 0:
            # then we look at the max distance, gather those peptides, randomly choose one
            # find associated CDR3s that don't overlap with the current peptide
            vmax = pep_levenshtein.max()
            irr_peps = pep_levenshtein.index[pep_levenshtein == vmax]
            # reset for the next round if needed
            pep_levenshtein = pep_levenshtein[pep_levenshtein < vmax]
            # find the irrelevant peptide CDR3s and make sure they don't overlap
            mask = (train_orig['Peptide'].isin(irr_peps)) & (~train_orig['CDR3b_extended'].isin(pep_cdr3s))
            if sum(mask) == 0:
                continue
            # if we have cdr3s then grab them out
            cdr3s = train_orig.loc[mask, 'CDR3b_extended']
            # if there are more than we need randomly select
            if len(cdr3s) > n_cdr3s:
                irr_cdr3s += np.random.choice(cdr3s, size=n_cdr3s, replace=False).tolist()
                break
            # otherwise keep moving
            else:
                irr_cdr3s += cdr3s.tolist()
                n_cdr3s -= len(cdr3s)
        # compile the full list
        irr = pd.DataFrame(irr_cdr3s, columns=['CDR3b_extended'])
        irr['Peptide'] = pep
        irrs.append(irr)

    from sklearn.metrics import roc_curve, auc, accuracy_score

    ## CONVERT TO CORRECT FORMAT
    # create X for training
    X_train = pd.concat([pd.concat(irrs, axis=0), train], axis=0)
    X_train_cdr3s = cdr3_to_X.loc[X_train['CDR3b_extended']].reset_index().iloc[:, 1:]
    X_train_epitopes = ag_to_X.loc[X_train['Peptide']].reset_index().iloc[:, 1:]
    # combine
    X_train_cdr3s.columns = 'cdr3:' + X_train_cdr3s.columns
    X_train_epitopes.columns = 'ag:' + X_train_epitopes.columns
    X_train = X_train_cdr3s.join(X_train_epitopes)

    # grab y for training
    y_train = pd.Series(0, index=X_train.index)
    y_train.iloc[-train.shape[0]:] = 1

    # confirm the same length
    assert X_train.shape[0] == y_train.shape[0]

    # create X for testing
    X_test_cdr3s = cdr3_to_X.loc[test['CDR3b_extended']].reset_index().iloc[:, 1:]
    X_test_epitopes = ag_to_X.loc[test['Peptide']].reset_index().iloc[:, 1:]
    # combine
    X_test_cdr3s.columns = 'cdr3:' + X_test_cdr3s.columns
    X_test_epitopes.columns = 'ag:' + X_test_epitopes.columns
    X_test = X_test_cdr3s.join(X_test_epitopes)

    # remove constant columns
    X_train = X_train.loc[:, X_train.nunique(0) > 1]
    X_train = X_train.loc[:, X_train.sum(0) > 0]
    # read in normalization factors
    means = X_train.mean(0)
    stds = (X_train - means).std(0)
    # subset for relevant columns
    X_train = X_train[means.index]
    X_test = X_test[means.index]
    # normalize
    X_train -= means
    X_train /= stds
    X_test -= means
    X_test /= stds


    ## SETUP MODEL
    # retrieve the appropriate columns
    cols_cdr3 = X_train.columns[X_train.columns.str.startswith('cdr3')]
    cols_ag = X_train.columns[X_train.columns.str.startswith('ag')]

    # determine model parameters
    # > layer for cdr3 alone
    input_1 = keras.layers.Input(shape=(len(cols_cdr3)))
    output_1 = keras.layers.Dense(400, activation='sigmoid')(input_1)
    # > layer for ag alone
    input_2 = keras.layers.Input(shape=(len(cols_ag)))
    output_2 = keras.layers.Dense(200, activation='sigmoid')(input_2)
    # > combined layer
    concat_3 = keras.layers.Concatenate()([output_1, output_2])
    output_3 = keras.layers.Dense(200, activation='sigmoid')(concat_3)
    # > final logit softmax layer
    output_4 = keras.layers.Dense(1, activation='sigmoid')(output_3)
    model = keras.Model(inputs=[input_1, input_2], outputs=[output_4])
    # set up the training parameters for the model
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss='binary_crossentropy',
        metrics=['accuracy','AUC'],
    )
    # train the model
    history = model.fit([X_train[cols_cdr3], X_train[cols_ag]], y_train,
                        epochs=10,
                        validation_data=([X_train[cols_cdr3], X_train[cols_ag]], y_train),
                        workers=40, use_multiprocessing=True)

    # testing predictions
    y_pred = model.predict([X_test[cols_cdr3], X_test[cols_ag]],
                           workers=40, use_multiprocessing=True)[:, 0]
    y_pred = pd.Series(y_pred, index=test.index, name='Pred%')
    hla2pred[hla] = y_pred

In [ ]:
# add back in predictions
test = test_orig.copy()
test['Prediction'] = np.nan
for pred in hla2pred.values():
    test.loc[pred.index, 'Prediction'] = pred
# write down predictions
test.to_csv('../external/tcr-specificity-prediction-challenge/submission.csv', index=False)

## tcrb only v5

In [ ]:
# retrieve hla specific predictions
hla2pred = {}
for hla in test_orig['HLA'].unique():
    # subset for HLA
    train = train_orig.loc[train_orig['HLA'] == hla].copy()
    test = test_orig.loc[test_orig['HLA'] == hla].copy()

    from tqdm import tqdm
    # set seed and identify irrelevant matches
    np.random.seed(0)
    irrs = []

    # re count the peptides for this HLA
    counts = train['Peptide'].value_counts()
    for pep in tqdm(counts.index):
        # gather peptide cdr3 information
        n_cdr3s = counts.loc[pep]
        pep_cdr3s = train.loc[train['Peptide'] == pep, 'CDR3b_extended'].unique()
        # we first grab the peptide levenshtein distances
        pep_levenshtein = train_l[pep].sort_values()[::-1]
        # systematically identify negative controls
        irr_cdr3s = []
        while n_cdr3s > 0:
            # then we look at the max distance, gather those peptides, randomly choose one
            # find associated CDR3s that don't overlap with the current peptide
            vmax = pep_levenshtein.max()
            irr_peps = pep_levenshtein.index[pep_levenshtein == vmax]
            # reset for the next round if needed
            pep_levenshtein = pep_levenshtein[pep_levenshtein < vmax]
            # find the irrelevant peptide CDR3s and make sure they don't overlap
            mask = (train_orig['Peptide'].isin(irr_peps)) & (~train_orig['CDR3b_extended'].isin(pep_cdr3s))
            if sum(mask) == 0:
                continue
            # if we have cdr3s then grab them out
            cdr3s = train_orig.loc[mask, 'CDR3b_extended']
            # if there are more than we need randomly select
            if len(cdr3s) > n_cdr3s:
                irr_cdr3s += np.random.choice(cdr3s, size=n_cdr3s, replace=False).tolist()
                break
            # otherwise keep moving
            else:
                irr_cdr3s += cdr3s.tolist()
                n_cdr3s -= len(cdr3s)
        # compile the full list
        irr = pd.DataFrame(irr_cdr3s, columns=['CDR3b_extended'])
        irr['Peptide'] = pep
        irrs.append(irr)

    from sklearn.metrics import roc_curve, auc, accuracy_score

    ## CONVERT TO CORRECT FORMAT
    # create X for training
    X_train = pd.concat([pd.concat(irrs, axis=0), train], axis=0)
    X_train_cdr3s = cdr3_to_X.loc[X_train['CDR3b_extended']].reset_index().iloc[:, 1:]
    X_train_epitopes = ag_to_X.loc[X_train['Peptide']].reset_index().iloc[:, 1:]
    # combine
    X_train_cdr3s.columns = 'cdr3:' + X_train_cdr3s.columns
    X_train_epitopes.columns = 'ag:' + X_train_epitopes.columns
    X_train = X_train_cdr3s.join(X_train_epitopes)

    # grab y for training
    y_train = pd.Series(0, index=X_train.index)
    y_train.iloc[-train.shape[0]:] = 1

    # confirm the same length
    assert X_train.shape[0] == y_train.shape[0]

    # create X for testing
    X_test_cdr3s = cdr3_to_X.loc[test['CDR3b_extended']].reset_index().iloc[:, 1:]
    X_test_epitopes = ag_to_X.loc[test['Peptide']].reset_index().iloc[:, 1:]
    # combine
    X_test_cdr3s.columns = 'cdr3:' + X_test_cdr3s.columns
    X_test_epitopes.columns = 'ag:' + X_test_epitopes.columns
    X_test = X_test_cdr3s.join(X_test_epitopes)

    # remove constant columns
    X_train = X_train.loc[:, X_train.nunique(0) > 1]
    X_train = X_train.loc[:, X_train.sum(0) > 0]
    # read in normalization factors
    means = X_train.mean(0)
    stds = (X_train - means).std(0)
    # subset for relevant columns
    X_train = X_train[means.index]
    X_test = X_test[means.index]
    # normalize
    X_train -= means
    X_train /= stds
    X_test -= means
    X_test /= stds


    ## SETUP MODEL
    # retrieve the appropriate columns
    cols_cdr3 = X_train.columns[X_train.columns.str.startswith('cdr3')]
    cols_ag = X_train.columns[X_train.columns.str.startswith('ag')]

    # determine model parameters
    # > layer for cdr3 alone
    input_1 = keras.layers.Input(shape=(len(cols_cdr3)))
    output_1 = keras.layers.Dense(1000, activation='sigmoid')(input_1)
    # > layer for ag alone
    input_2 = keras.layers.Input(shape=(len(cols_ag)))
    output_2 = keras.layers.Dense(500, activation='sigmoid')(input_2)
    # > combined layer
    concat_3 = keras.layers.Concatenate()([output_1, output_2])
    output_3 = keras.layers.Dense(500, activation='sigmoid')(concat_3)
    # > final logit softmax layer
    output_4 = keras.layers.Dense(1, activation='sigmoid')(output_3)
    model = keras.Model(inputs=[input_1, input_2], outputs=[output_4])
    # set up the training parameters for the model
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss='binary_crossentropy',
        metrics=['accuracy','AUC'],
    )
    # train the model
    history = model.fit([X_train[cols_cdr3], X_train[cols_ag]], y_train,
                        epochs=10,
                        validation_data=([X_train[cols_cdr3], X_train[cols_ag]], y_train),
                        workers=40, use_multiprocessing=True)

    # testing predictions
    y_pred = model.predict([X_test[cols_cdr3], X_test[cols_ag]],
                           workers=40, use_multiprocessing=True)[:, 0]
    y_pred = pd.Series(y_pred, index=test.index, name='Pred%')
    hla2pred[hla] = y_pred

In [ ]:
# add back in predictions
test = test_orig.copy()
test['Prediction'] = np.nan
for pred in hla2pred.values():
    test.loc[pred.index, 'Prediction'] = pred
# write down predictions
test.to_csv('../external/tcr-specificity-prediction-challenge/submission.csv', index=False)

## tcra and tcrb only v1

In [ ]:
# retrieve encodings
cdr3a_to_X = encode_cdr3s(pd.concat([train_orig, test_orig], axis=0)['CDR3a_extended'].dropna().unique(), 40, False)
cdr3b_to_X = encode_cdr3s(pd.concat([train_orig, test_orig], axis=0)['CDR3b_extended'].dropna().unique(), 40, False)

In [ ]:
# retrieve hla specific predictions
hla2pred = {}
for hla in test_orig['HLA'].unique():
    # subset for HLA
    train = train_orig.loc[train_orig['HLA'] == hla].copy()
    test = test_orig.loc[test_orig['HLA'] == hla].copy()
    train = train.dropna(subset=['CDR3a_extended'])

    from tqdm import tqdm
    # set seed and identify irrelevant matches
    np.random.seed(0)
    irrs = []

    # re count the peptides for this HLA
    counts = train['Peptide'].value_counts()
    for pep in tqdm(counts.index):
        # gather peptide cdr3 information
        n_cdr3s = counts.loc[pep]
        pep_cdr3s = train.loc[train['Peptide'] == pep, ['CDR3a_extended','CDR3b_extended']].value_counts().reset_index()
        # we first grab the peptide levenshtein distances
        pep_levenshtein = train_l[pep].sort_values()[::-1]
        # systematically identify negative controls
        irr_cdr3s = []
        while n_cdr3s > 0:
            # then we look at the max distance, gather those peptides, randomly choose one
            # find associated CDR3s that don't overlap with the current peptide
            vmax = pep_levenshtein.max()
            irr_peps = pep_levenshtein.index[pep_levenshtein == vmax]
            # reset for the next round if needed
            pep_levenshtein = pep_levenshtein[pep_levenshtein < vmax]
            # find the irrelevant peptide CDR3s and make sure they don't overlap
            mask = (train_orig['Peptide'].isin(irr_peps)) &\
            ((~train_orig['CDR3a_extended'].isin(pep_cdr3s['CDR3a_extended'])) |\
            (~train_orig['CDR3b_extended'].isin(pep_cdr3s['CDR3b_extended']))) &\
            (~train_orig['CDR3a_extended'].isna()) &\
            (~train_orig['CDR3b_extended'].isna())
            if sum(mask) == 0:
                continue
            # if we have cdr3s then grab them out
            cdr3s = train_orig.loc[mask, ['CDR3a_extended','CDR3b_extended']]
            # if there are more than we need randomly select
            if len(cdr3s) > n_cdr3s:
                irr_cdr3s_idxs = np.random.choice(cdr3s.index, size=n_cdr3s, replace=False)
                irr_cdr3s += cdr3s.loc[irr_cdr3s_idxs].values.tolist()
                break
            # otherwise keep moving
            else:
                irr_cdr3s += cdr3s.values.tolist()
                n_cdr3s -= len(cdr3s)
        # compile the full list
        irr = pd.DataFrame(irr_cdr3s, columns=['CDR3a_extended','CDR3b_extended'])
        irr['Peptide'] = pep
        irrs.append(irr)

    from sklearn.metrics import roc_curve, auc, accuracy_score

    ## CONVERT TO CORRECT FORMAT
    # create X for training
    X_train = pd.concat([pd.concat(irrs, axis=0), train], axis=0)
    X_train_cdr3as = cdr3a_to_X.loc[X_train['CDR3a_extended']].reset_index().iloc[:, 1:]
    X_train_cdr3bs = cdr3b_to_X.loc[X_train['CDR3b_extended']].reset_index().iloc[:, 1:]
    X_train_epitopes = ag_to_X.loc[X_train['Peptide']].reset_index().iloc[:, 1:]
    # combine
    X_train_cdr3as.columns = 'cdr3a:' + X_train_cdr3as.columns
    X_train_cdr3bs.columns = 'cdr3b:' + X_train_cdr3bs.columns
    X_train_epitopes.columns = 'ag:' + X_train_epitopes.columns
    X_train = X_train_cdr3as.join(X_train_cdr3bs).join(X_train_epitopes)

    # grab y for training
    y_train = pd.Series(0, index=X_train.index)
    y_train.iloc[-train.shape[0]:] = 1

    # confirm the same length
    assert X_train.shape[0] == y_train.shape[0]

    # create X for testing
    X_test_cdr3as = cdr3a_to_X.loc[test['CDR3a_extended']].reset_index().iloc[:, 1:]
    X_test_cdr3bs = cdr3b_to_X.loc[test['CDR3b_extended']].reset_index().iloc[:, 1:]
    X_test_epitopes = ag_to_X.loc[test['Peptide']].reset_index().iloc[:, 1:]
    # combine
    X_test_cdr3as.columns = 'cdr3a:' + X_test_cdr3as.columns
    X_test_cdr3bs.columns = 'cdr3b:' + X_test_cdr3bs.columns
    X_test_epitopes.columns = 'ag:' + X_test_epitopes.columns
    X_test = X_test_cdr3as.join(X_test_cdr3bs).join(X_test_epitopes)

    # remove constant columns
    X_train = X_train.loc[:, X_train.nunique(0) > 1]
    X_train = X_train.loc[:, X_train.sum(0) > 0]
    # read in normalization factors
    means = X_train.mean(0)
    stds = (X_train - means).std(0)
    # subset for relevant columns
    X_train = X_train[means.index]
    X_test = X_test[means.index]
    # normalize
    X_train -= means
    X_train /= stds
    X_test -= means
    X_test /= stds


    ## SETUP MODEL
    # retrieve the appropriate columns
    cols_cdr3a = X_train.columns[X_train.columns.str.startswith('cdr3a')]
    cols_cdr3b = X_train.columns[X_train.columns.str.startswith('cdr3b')]
    cols_ag = X_train.columns[X_train.columns.str.startswith('ag')]

    # determine model parameters
    # > layer for cdr3a alone
    input_0 = keras.layers.Input(shape=(len(cols_cdr3a)))
    output_0 = keras.layers.Dense(200, activation='sigmoid')(input_0)
    # > layer for cdr3b alone
    input_1 = keras.layers.Input(shape=(len(cols_cdr3b)))
    output_1 = keras.layers.Dense(200, activation='sigmoid')(input_1)
    # > layer for ag alone
    input_2 = keras.layers.Input(shape=(len(cols_ag)))
    output_2 = keras.layers.Dense(100, activation='sigmoid')(input_2)
    # > combined layer for tcra and tcrb
    concat_3 = keras.layers.Concatenate()([output_0, output_1])
    output_3 = keras.layers.Dense(100, activation='sigmoid')(concat_3)
    # > combined layer for tcr and ag
    concat_4 = keras.layers.Concatenate()([output_2, output_3])
    output_4 = keras.layers.Dense(100, activation='sigmoid')(concat_4)
    # > final logit softmax layer
    output_5 = keras.layers.Dense(1, activation='sigmoid')(output_4)
    model = keras.Model(inputs=[input_0, input_1, input_2], outputs=[output_5])
    # set up the training parameters for the model
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss='binary_crossentropy',
        metrics=['accuracy','AUC'],
    )
    # train the model
    history = model.fit([X_train[cols_cdr3a], X_train[cols_cdr3b], X_train[cols_ag]], y_train,
                        epochs=10,
                        validation_data=([X_train[cols_cdr3a], X_train[cols_cdr3b], X_train[cols_ag]], y_train),
                        workers=40, use_multiprocessing=True)

    # testing predictions
    y_pred = model.predict([X_test[cols_cdr3a], X_test[cols_cdr3b], X_test[cols_ag]],
                           workers=40, use_multiprocessing=True)[:, 0]
    y_pred = pd.Series(y_pred, index=test.index, name='Pred%')

    hla2pred[hla] = y_pred

In [ ]:
# add back in predictions
test = test_orig.copy()
test['Prediction'] = np.nan
for pred in hla2pred.values():
    test.loc[pred.index, 'Prediction'] = pred
# write down predictions
test.to_csv('../external/tcr-specificity-prediction-challenge/submission.csv', index=False)

## tcra and tcrb only v2

In [ ]:
# retrieve hla specific predictions
hla2pred = {}
for hla in test_orig['HLA'].unique():
    # subset for HLA
    train = train_orig.loc[train_orig['HLA'] == hla].copy()
    test = test_orig.loc[test_orig['HLA'] == hla].copy()
    train = train.dropna(subset=['CDR3a_extended'])

    from tqdm import tqdm
    # set seed and identify irrelevant matches
    np.random.seed(0)
    irrs = []

    # re count the peptides for this HLA
    counts = train['Peptide'].value_counts()
    for pep in tqdm(counts.index):
        # gather peptide cdr3 information
        n_cdr3s = counts.loc[pep]
        pep_cdr3s = train.loc[train['Peptide'] == pep, ['CDR3a_extended','CDR3b_extended']].value_counts().reset_index()
        # we first grab the peptide levenshtein distances
        pep_levenshtein = train_l[pep].sort_values()[::-1]
        # systematically identify negative controls
        irr_cdr3s = []
        while n_cdr3s > 0:
            # then we look at the max distance, gather those peptides, randomly choose one
            # find associated CDR3s that don't overlap with the current peptide
            vmax = pep_levenshtein.max()
            irr_peps = pep_levenshtein.index[pep_levenshtein == vmax]
            # reset for the next round if needed
            pep_levenshtein = pep_levenshtein[pep_levenshtein < vmax]
            # find the irrelevant peptide CDR3s and make sure they don't overlap
            mask = (train_orig['Peptide'].isin(irr_peps)) &\
            ((~train_orig['CDR3a_extended'].isin(pep_cdr3s['CDR3a_extended'])) |\
            (~train_orig['CDR3b_extended'].isin(pep_cdr3s['CDR3b_extended']))) &\
            (~train_orig['CDR3a_extended'].isna()) &\
            (~train_orig['CDR3b_extended'].isna())
            if sum(mask) == 0:
                continue
            # if we have cdr3s then grab them out
            cdr3s = train_orig.loc[mask, ['CDR3a_extended','CDR3b_extended']]
            # if there are more than we need randomly select
            if len(cdr3s) > n_cdr3s:
                irr_cdr3s_idxs = np.random.choice(cdr3s.index, size=n_cdr3s, replace=False)
                irr_cdr3s += cdr3s.loc[irr_cdr3s_idxs].values.tolist()
                break
            # otherwise keep moving
            else:
                irr_cdr3s += cdr3s.values.tolist()
                n_cdr3s -= len(cdr3s)
        # compile the full list
        irr = pd.DataFrame(irr_cdr3s, columns=['CDR3a_extended','CDR3b_extended'])
        irr['Peptide'] = pep
        irrs.append(irr)

    from sklearn.metrics import roc_curve, auc, accuracy_score

    ## CONVERT TO CORRECT FORMAT
    # create X for training
    X_train = pd.concat([pd.concat(irrs, axis=0), train], axis=0)
    X_train_cdr3as = cdr3a_to_X.loc[X_train['CDR3a_extended']].reset_index().iloc[:, 1:]
    X_train_cdr3bs = cdr3b_to_X.loc[X_train['CDR3b_extended']].reset_index().iloc[:, 1:]
    X_train_epitopes = ag_to_X.loc[X_train['Peptide']].reset_index().iloc[:, 1:]
    # combine
    X_train_cdr3as.columns = 'cdr3a:' + X_train_cdr3as.columns
    X_train_cdr3bs.columns = 'cdr3b:' + X_train_cdr3bs.columns
    X_train_epitopes.columns = 'ag:' + X_train_epitopes.columns
    X_train = X_train_cdr3as.join(X_train_cdr3bs).join(X_train_epitopes)

    # grab y for training
    y_train = pd.Series(0, index=X_train.index)
    y_train.iloc[-train.shape[0]:] = 1

    # confirm the same length
    assert X_train.shape[0] == y_train.shape[0]

    # create X for testing
    X_test_cdr3as = cdr3a_to_X.loc[test['CDR3a_extended']].reset_index().iloc[:, 1:]
    X_test_cdr3bs = cdr3b_to_X.loc[test['CDR3b_extended']].reset_index().iloc[:, 1:]
    X_test_epitopes = ag_to_X.loc[test['Peptide']].reset_index().iloc[:, 1:]
    # combine
    X_test_cdr3as.columns = 'cdr3a:' + X_test_cdr3as.columns
    X_test_cdr3bs.columns = 'cdr3b:' + X_test_cdr3bs.columns
    X_test_epitopes.columns = 'ag:' + X_test_epitopes.columns
    X_test = X_test_cdr3as.join(X_test_cdr3bs).join(X_test_epitopes)

    # remove constant columns
    X_train = X_train.loc[:, X_train.nunique(0) > 1]
    X_train = X_train.loc[:, X_train.sum(0) > 0]
    # read in normalization factors
    means = X_train.mean(0)
    stds = (X_train - means).std(0)
    # subset for relevant columns
    X_train = X_train[means.index]
    X_test = X_test[means.index]
    # normalize
    X_train -= means
    X_train /= stds
    X_test -= means
    X_test /= stds


    ## SETUP MODEL
    # retrieve the appropriate columns
    cols_cdr3a = X_train.columns[X_train.columns.str.startswith('cdr3a')]
    cols_cdr3b = X_train.columns[X_train.columns.str.startswith('cdr3b')]
    cols_ag = X_train.columns[X_train.columns.str.startswith('ag')]

    # determine model parameters
    # > layer for cdr3a alone
    input_0 = keras.layers.Input(shape=(len(cols_cdr3a)))
    output_0 = keras.layers.Dense(300, activation='sigmoid')(input_0)
    # > layer for cdr3b alone
    input_1 = keras.layers.Input(shape=(len(cols_cdr3b)))
    output_1 = keras.layers.Dense(300, activation='sigmoid')(input_1)
    # > layer for ag alone
    input_2 = keras.layers.Input(shape=(len(cols_ag)))
    output_2 = keras.layers.Dense(200, activation='sigmoid')(input_2)
    # > combined layer for tcra and tcrb
    concat_3 = keras.layers.Concatenate()([output_0, output_1])
    output_3 = keras.layers.Dense(100, activation='sigmoid')(concat_3)
    # > combined layer for tcr and ag
    concat_4 = keras.layers.Concatenate()([output_2, output_3])
    output_4 = keras.layers.Dense(100, activation='sigmoid')(concat_4)
    # > final logit softmax layer
    output_5 = keras.layers.Dense(1, activation='sigmoid')(output_4)
    model = keras.Model(inputs=[input_0, input_1, input_2], outputs=[output_5])
    # set up the training parameters for the model
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss='binary_crossentropy',
        metrics=['accuracy','AUC'],
    )
    # train the model
    history = model.fit([X_train[cols_cdr3a], X_train[cols_cdr3b], X_train[cols_ag]], y_train,
                        epochs=10,
                        validation_data=([X_train[cols_cdr3a], X_train[cols_cdr3b], X_train[cols_ag]], y_train),
                        workers=40, use_multiprocessing=True)

    # testing predictions
    y_pred = model.predict([X_test[cols_cdr3a], X_test[cols_cdr3b], X_test[cols_ag]],
                           workers=40, use_multiprocessing=True)[:, 0]
    y_pred = pd.Series(y_pred, index=test.index, name='Pred%')

    hla2pred[hla] = y_pred

In [ ]:
# add back in predictions
test = test_orig.copy()
test['Prediction'] = np.nan
for pred in hla2pred.values():
    test.loc[pred.index, 'Prediction'] = pred
# write down predictions
test.to_csv('../external/tcr-specificity-prediction-challenge/submission.csv', index=False)

## tcra and tcrb only v3

In [ ]:
# retrieve hla specific predictions
hla2pred = {}
for hla in test_orig['HLA'].unique():
    # subset for HLA
    train = train_orig.loc[train_orig['HLA'] == hla].copy()
    test = test_orig.loc[test_orig['HLA'] == hla].copy()
    train = train.dropna(subset=['CDR3a_extended'])

    from tqdm import tqdm
    # set seed and identify irrelevant matches
    np.random.seed(0)
    irrs = []

    # re count the peptides for this HLA
    counts = train['Peptide'].value_counts()
    for pep in tqdm(counts.index):
        # gather peptide cdr3 information
        n_cdr3s = counts.loc[pep]
        pep_cdr3s = train.loc[train['Peptide'] == pep, ['CDR3a_extended','CDR3b_extended']].value_counts().reset_index()
        # we first grab the peptide levenshtein distances
        pep_levenshtein = train_l[pep].sort_values()[::-1]
        # systematically identify negative controls
        irr_cdr3s = []
        while n_cdr3s > 0:
            # then we look at the max distance, gather those peptides, randomly choose one
            # find associated CDR3s that don't overlap with the current peptide
            vmax = pep_levenshtein.max()
            irr_peps = pep_levenshtein.index[pep_levenshtein == vmax]
            # reset for the next round if needed
            pep_levenshtein = pep_levenshtein[pep_levenshtein < vmax]
            # find the irrelevant peptide CDR3s and make sure they don't overlap
            mask = (train_orig['Peptide'].isin(irr_peps)) &\
            ((~train_orig['CDR3a_extended'].isin(pep_cdr3s['CDR3a_extended'])) |\
            (~train_orig['CDR3b_extended'].isin(pep_cdr3s['CDR3b_extended']))) &\
            (~train_orig['CDR3a_extended'].isna()) &\
            (~train_orig['CDR3b_extended'].isna())
            if sum(mask) == 0:
                continue
            # if we have cdr3s then grab them out
            cdr3s = train_orig.loc[mask, ['CDR3a_extended','CDR3b_extended']]
            # if there are more than we need randomly select
            if len(cdr3s) > n_cdr3s:
                irr_cdr3s_idxs = np.random.choice(cdr3s.index, size=n_cdr3s, replace=False)
                irr_cdr3s += cdr3s.loc[irr_cdr3s_idxs].values.tolist()
                break
            # otherwise keep moving
            else:
                irr_cdr3s += cdr3s.values.tolist()
                n_cdr3s -= len(cdr3s)
        # compile the full list
        irr = pd.DataFrame(irr_cdr3s, columns=['CDR3a_extended','CDR3b_extended'])
        irr['Peptide'] = pep
        irrs.append(irr)

    from sklearn.metrics import roc_curve, auc, accuracy_score

    ## CONVERT TO CORRECT FORMAT
    # create X for training
    X_train = pd.concat([pd.concat(irrs, axis=0), train], axis=0)
    X_train_cdr3as = cdr3a_to_X.loc[X_train['CDR3a_extended']].reset_index().iloc[:, 1:]
    X_train_cdr3bs = cdr3b_to_X.loc[X_train['CDR3b_extended']].reset_index().iloc[:, 1:]
    X_train_epitopes = ag_to_X.loc[X_train['Peptide']].reset_index().iloc[:, 1:]
    # combine
    X_train_cdr3as.columns = 'cdr3a:' + X_train_cdr3as.columns
    X_train_cdr3bs.columns = 'cdr3b:' + X_train_cdr3bs.columns
    X_train_epitopes.columns = 'ag:' + X_train_epitopes.columns
    X_train = X_train_cdr3as.join(X_train_cdr3bs).join(X_train_epitopes)

    # grab y for training
    y_train = pd.Series(0, index=X_train.index)
    y_train.iloc[-train.shape[0]:] = 1

    # confirm the same length
    assert X_train.shape[0] == y_train.shape[0]

    # create X for testing
    X_test_cdr3as = cdr3a_to_X.loc[test['CDR3a_extended']].reset_index().iloc[:, 1:]
    X_test_cdr3bs = cdr3b_to_X.loc[test['CDR3b_extended']].reset_index().iloc[:, 1:]
    X_test_epitopes = ag_to_X.loc[test['Peptide']].reset_index().iloc[:, 1:]
    # combine
    X_test_cdr3as.columns = 'cdr3a:' + X_test_cdr3as.columns
    X_test_cdr3bs.columns = 'cdr3b:' + X_test_cdr3bs.columns
    X_test_epitopes.columns = 'ag:' + X_test_epitopes.columns
    X_test = X_test_cdr3as.join(X_test_cdr3bs).join(X_test_epitopes)

    # remove constant columns
    X_train = X_train.loc[:, X_train.nunique(0) > 1]
    X_train = X_train.loc[:, X_train.sum(0) > 0]
    # read in normalization factors
    means = X_train.mean(0)
    stds = (X_train - means).std(0)
    # subset for relevant columns
    X_train = X_train[means.index]
    X_test = X_test[means.index]
    # normalize
    X_train -= means
    X_train /= stds
    X_test -= means
    X_test /= stds


    ## SETUP MODEL
    # retrieve the appropriate columns
    cols_cdr3a = X_train.columns[X_train.columns.str.startswith('cdr3a')]
    cols_cdr3b = X_train.columns[X_train.columns.str.startswith('cdr3b')]
    cols_ag = X_train.columns[X_train.columns.str.startswith('ag')]

    # determine model parameters
    # > layer for cdr3a alone
    input_0 = keras.layers.Input(shape=(len(cols_cdr3a)))
    output_0 = keras.layers.Dense(300, activation='sigmoid')(input_0)
    # > layer for cdr3b alone
    input_1 = keras.layers.Input(shape=(len(cols_cdr3b)))
    output_1 = keras.layers.Dense(300, activation='sigmoid')(input_1)
    # > layer for ag alone
    input_2 = keras.layers.Input(shape=(len(cols_ag)))
    output_2 = keras.layers.Dense(100, activation='sigmoid')(input_2)
    # > combined layer for tcra and tcrb
    concat_3 = keras.layers.Concatenate()([output_0, output_1])
    output_3 = keras.layers.Dense(200, activation='sigmoid')(concat_3)
    # > combined layer for tcr and ag
    concat_4 = keras.layers.Concatenate()([output_2, output_3])
    output_4 = keras.layers.Dense(100, activation='sigmoid')(concat_4)
    # > final logit softmax layer
    output_5 = keras.layers.Dense(1, activation='sigmoid')(output_4)
    model = keras.Model(inputs=[input_0, input_1, input_2], outputs=[output_5])
    # set up the training parameters for the model
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss='binary_crossentropy',
        metrics=['accuracy','AUC'],
    )
    # train the model
    history = model.fit([X_train[cols_cdr3a], X_train[cols_cdr3b], X_train[cols_ag]], y_train,
                        epochs=10,
                        validation_data=([X_train[cols_cdr3a], X_train[cols_cdr3b], X_train[cols_ag]], y_train),
                        workers=40, use_multiprocessing=True)

    # testing predictions
    y_pred = model.predict([X_test[cols_cdr3a], X_test[cols_cdr3b], X_test[cols_ag]],
                           workers=40, use_multiprocessing=True)[:, 0]
    y_pred = pd.Series(y_pred, index=test.index, name='Pred%')

    hla2pred[hla] = y_pred

In [ ]:
# add back in predictions
test = test_orig.copy()
test['Prediction'] = np.nan
for pred in hla2pred.values():
    test.loc[pred.index, 'Prediction'] = pred
# write down predictions
test.to_csv('../external/tcr-specificity-prediction-challenge/submission.csv', index=False)

## tcra and tcrb only v4

In [ ]:
# retrieve hla specific predictions
hla2pred = {}
for hla in test_orig['HLA'].unique():
    # subset for HLA
    train = train_orig.loc[train_orig['HLA'] == hla].copy()
    test = test_orig.loc[test_orig['HLA'] == hla].copy()
    train = train.dropna(subset=['CDR3a_extended'])

    from tqdm import tqdm
    # set seed and identify irrelevant matches
    np.random.seed(0)
    irrs = []

    # re count the peptides for this HLA
    counts = train['Peptide'].value_counts()
    for pep in tqdm(counts.index):
        # gather peptide cdr3 information
        n_cdr3s = counts.loc[pep]
        pep_cdr3s = train.loc[train['Peptide'] == pep, ['CDR3a_extended','CDR3b_extended']].value_counts().reset_index()
        # we first grab the peptide levenshtein distances
        pep_levenshtein = train_l[pep].sort_values()[::-1]
        # systematically identify negative controls
        irr_cdr3s = []
        while n_cdr3s > 0:
            # then we look at the max distance, gather those peptides, randomly choose one
            # find associated CDR3s that don't overlap with the current peptide
            vmax = pep_levenshtein.max()
            irr_peps = pep_levenshtein.index[pep_levenshtein == vmax]
            # reset for the next round if needed
            pep_levenshtein = pep_levenshtein[pep_levenshtein < vmax]
            # find the irrelevant peptide CDR3s and make sure they don't overlap
            mask = (train_orig['Peptide'].isin(irr_peps)) &\
            ((~train_orig['CDR3a_extended'].isin(pep_cdr3s['CDR3a_extended'])) |\
            (~train_orig['CDR3b_extended'].isin(pep_cdr3s['CDR3b_extended']))) &\
            (~train_orig['CDR3a_extended'].isna()) &\
            (~train_orig['CDR3b_extended'].isna())
            if sum(mask) == 0:
                continue
            # if we have cdr3s then grab them out
            cdr3s = train_orig.loc[mask, ['CDR3a_extended','CDR3b_extended']]
            # if there are more than we need randomly select
            if len(cdr3s) > n_cdr3s:
                irr_cdr3s_idxs = np.random.choice(cdr3s.index, size=n_cdr3s, replace=False)
                irr_cdr3s += cdr3s.loc[irr_cdr3s_idxs].values.tolist()
                break
            # otherwise keep moving
            else:
                irr_cdr3s += cdr3s.values.tolist()
                n_cdr3s -= len(cdr3s)
        # compile the full list
        irr = pd.DataFrame(irr_cdr3s, columns=['CDR3a_extended','CDR3b_extended'])
        irr['Peptide'] = pep
        irrs.append(irr)

    from sklearn.metrics import roc_curve, auc, accuracy_score

    ## CONVERT TO CORRECT FORMAT
    # create X for training
    X_train = pd.concat([pd.concat(irrs, axis=0), train], axis=0)
    X_train_cdr3as = cdr3a_to_X.loc[X_train['CDR3a_extended']].reset_index().iloc[:, 1:]
    X_train_cdr3bs = cdr3b_to_X.loc[X_train['CDR3b_extended']].reset_index().iloc[:, 1:]
    X_train_epitopes = ag_to_X.loc[X_train['Peptide']].reset_index().iloc[:, 1:]
    # combine
    X_train_cdr3as.columns = 'cdr3a:' + X_train_cdr3as.columns
    X_train_cdr3bs.columns = 'cdr3b:' + X_train_cdr3bs.columns
    X_train_epitopes.columns = 'ag:' + X_train_epitopes.columns
    X_train = X_train_cdr3as.join(X_train_cdr3bs).join(X_train_epitopes)

    # grab y for training
    y_train = pd.Series(0, index=X_train.index)
    y_train.iloc[-train.shape[0]:] = 1

    # confirm the same length
    assert X_train.shape[0] == y_train.shape[0]

    # create X for testing
    X_test_cdr3as = cdr3a_to_X.loc[test['CDR3a_extended']].reset_index().iloc[:, 1:]
    X_test_cdr3bs = cdr3b_to_X.loc[test['CDR3b_extended']].reset_index().iloc[:, 1:]
    X_test_epitopes = ag_to_X.loc[test['Peptide']].reset_index().iloc[:, 1:]
    # combine
    X_test_cdr3as.columns = 'cdr3a:' + X_test_cdr3as.columns
    X_test_cdr3bs.columns = 'cdr3b:' + X_test_cdr3bs.columns
    X_test_epitopes.columns = 'ag:' + X_test_epitopes.columns
    X_test = X_test_cdr3as.join(X_test_cdr3bs).join(X_test_epitopes)

    # remove constant columns
    X_train = X_train.loc[:, X_train.nunique(0) > 1]
    X_train = X_train.loc[:, X_train.sum(0) > 0]
    # read in normalization factors
    means = X_train.mean(0)
    stds = (X_train - means).std(0)
    # subset for relevant columns
    X_train = X_train[means.index]
    X_test = X_test[means.index]
    # normalize
    X_train -= means
    X_train /= stds
    X_test -= means
    X_test /= stds


    ## SETUP MODEL
    # retrieve the appropriate columns
    cols_cdr3a = X_train.columns[X_train.columns.str.startswith('cdr3a')]
    cols_cdr3b = X_train.columns[X_train.columns.str.startswith('cdr3b')]
    cols_ag = X_train.columns[X_train.columns.str.startswith('ag')]

    # determine model parameters
    # > layer for cdr3a alone
    input_0 = keras.layers.Input(shape=(len(cols_cdr3a)))
    output_0 = keras.layers.Dense(400, activation='sigmoid')(input_0)
    # > layer for cdr3b alone
    input_1 = keras.layers.Input(shape=(len(cols_cdr3b)))
    output_1 = keras.layers.Dense(400, activation='sigmoid')(input_1)
    # > layer for ag alone
    input_2 = keras.layers.Input(shape=(len(cols_ag)))
    output_2 = keras.layers.Dense(200, activation='sigmoid')(input_2)
    # > combined layer for tcra and tcrb
    concat_3 = keras.layers.Concatenate()([output_0, output_1])
    output_3 = keras.layers.Dense(300, activation='sigmoid')(concat_3)
    # > combined layer for tcr and ag
    concat_4 = keras.layers.Concatenate()([output_2, output_3])
    output_4 = keras.layers.Dense(200, activation='sigmoid')(concat_4)
    # > final logit softmax layer
    output_5 = keras.layers.Dense(1, activation='sigmoid')(output_4)
    model = keras.Model(inputs=[input_0, input_1, input_2], outputs=[output_5])
    # set up the training parameters for the model
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss='binary_crossentropy',
        metrics=['accuracy','AUC'],
    )
    # train the model
    history = model.fit([X_train[cols_cdr3a], X_train[cols_cdr3b], X_train[cols_ag]], y_train,
                        epochs=10,
                        validation_data=([X_train[cols_cdr3a], X_train[cols_cdr3b], X_train[cols_ag]], y_train),
                        workers=40, use_multiprocessing=True)

    # testing predictions
    y_pred = model.predict([X_test[cols_cdr3a], X_test[cols_cdr3b], X_test[cols_ag]],
                           workers=40, use_multiprocessing=True)[:, 0]
    y_pred = pd.Series(y_pred, index=test.index, name='Pred%')

    hla2pred[hla] = y_pred

In [ ]:
# add back in predictions
test = test_orig.copy()
test['Prediction'] = np.nan
for pred in hla2pred.values():
    test.loc[pred.index, 'Prediction'] = pred
# write down predictions
test.to_csv('../external/tcr-specificity-prediction-challenge/submission.csv', index=False)

## tcra and tcrb only v5

In [ ]:
# retrieve hla specific predictions
hla2pred = {}
for hla in test_orig['HLA'].unique():
    # subset for HLA
    train = train_orig.loc[train_orig['HLA'] == hla].copy()
    test = test_orig.loc[test_orig['HLA'] == hla].copy()
    train = train.dropna(subset=['CDR3a_extended'])

    from tqdm import tqdm
    # set seed and identify irrelevant matches
    np.random.seed(0)
    irrs = []

    # re count the peptides for this HLA
    counts = train['Peptide'].value_counts()
    for pep in tqdm(counts.index):
        # gather peptide cdr3 information
        n_cdr3s = counts.loc[pep]
        pep_cdr3s = train.loc[train['Peptide'] == pep, ['CDR3a_extended','CDR3b_extended']].value_counts().reset_index()
        # we first grab the peptide levenshtein distances
        pep_levenshtein = train_l[pep].sort_values()[::-1]
        # systematically identify negative controls
        irr_cdr3s = []
        while n_cdr3s > 0:
            # then we look at the max distance, gather those peptides, randomly choose one
            # find associated CDR3s that don't overlap with the current peptide
            vmax = pep_levenshtein.max()
            irr_peps = pep_levenshtein.index[pep_levenshtein == vmax]
            # reset for the next round if needed
            pep_levenshtein = pep_levenshtein[pep_levenshtein < vmax]
            # find the irrelevant peptide CDR3s and make sure they don't overlap
            mask = (train_orig['Peptide'].isin(irr_peps)) &\
            ((~train_orig['CDR3a_extended'].isin(pep_cdr3s['CDR3a_extended'])) |\
            (~train_orig['CDR3b_extended'].isin(pep_cdr3s['CDR3b_extended']))) &\
            (~train_orig['CDR3a_extended'].isna()) &\
            (~train_orig['CDR3b_extended'].isna())
            if sum(mask) == 0:
                continue
            # if we have cdr3s then grab them out
            cdr3s = train_orig.loc[mask, ['CDR3a_extended','CDR3b_extended']]
            # if there are more than we need randomly select
            if len(cdr3s) > n_cdr3s:
                irr_cdr3s_idxs = np.random.choice(cdr3s.index, size=n_cdr3s, replace=False)
                irr_cdr3s += cdr3s.loc[irr_cdr3s_idxs].values.tolist()
                break
            # otherwise keep moving
            else:
                irr_cdr3s += cdr3s.values.tolist()
                n_cdr3s -= len(cdr3s)
        # compile the full list
        irr = pd.DataFrame(irr_cdr3s, columns=['CDR3a_extended','CDR3b_extended'])
        irr['Peptide'] = pep
        irrs.append(irr)

    from sklearn.metrics import roc_curve, auc, accuracy_score

    ## CONVERT TO CORRECT FORMAT
    # create X for training
    X_train = pd.concat([pd.concat(irrs, axis=0), train], axis=0)
    X_train_cdr3as = cdr3a_to_X.loc[X_train['CDR3a_extended']].reset_index().iloc[:, 1:]
    X_train_cdr3bs = cdr3b_to_X.loc[X_train['CDR3b_extended']].reset_index().iloc[:, 1:]
    X_train_epitopes = ag_to_X.loc[X_train['Peptide']].reset_index().iloc[:, 1:]
    # combine
    X_train_cdr3as.columns = 'cdr3a:' + X_train_cdr3as.columns
    X_train_cdr3bs.columns = 'cdr3b:' + X_train_cdr3bs.columns
    X_train_epitopes.columns = 'ag:' + X_train_epitopes.columns
    X_train = X_train_cdr3as.join(X_train_cdr3bs).join(X_train_epitopes)

    # grab y for training
    y_train = pd.Series(0, index=X_train.index)
    y_train.iloc[-train.shape[0]:] = 1

    # confirm the same length
    assert X_train.shape[0] == y_train.shape[0]

    # create X for testing
    X_test_cdr3as = cdr3a_to_X.loc[test['CDR3a_extended']].reset_index().iloc[:, 1:]
    X_test_cdr3bs = cdr3b_to_X.loc[test['CDR3b_extended']].reset_index().iloc[:, 1:]
    X_test_epitopes = ag_to_X.loc[test['Peptide']].reset_index().iloc[:, 1:]
    # combine
    X_test_cdr3as.columns = 'cdr3a:' + X_test_cdr3as.columns
    X_test_cdr3bs.columns = 'cdr3b:' + X_test_cdr3bs.columns
    X_test_epitopes.columns = 'ag:' + X_test_epitopes.columns
    X_test = X_test_cdr3as.join(X_test_cdr3bs).join(X_test_epitopes)

    # remove constant columns
    X_train = X_train.loc[:, X_train.nunique(0) > 1]
    X_train = X_train.loc[:, X_train.sum(0) > 0]
    # read in normalization factors
    means = X_train.mean(0)
    stds = (X_train - means).std(0)
    # subset for relevant columns
    X_train = X_train[means.index]
    X_test = X_test[means.index]
    # normalize
    X_train -= means
    X_train /= stds
    X_test -= means
    X_test /= stds


    ## SETUP MODEL
    # retrieve the appropriate columns
    cols_cdr3a = X_train.columns[X_train.columns.str.startswith('cdr3a')]
    cols_cdr3b = X_train.columns[X_train.columns.str.startswith('cdr3b')]
    cols_ag = X_train.columns[X_train.columns.str.startswith('ag')]

    # determine model parameters
    # > layer for cdr3a alone
    input_0 = keras.layers.Input(shape=(len(cols_cdr3a)))
    output_0 = keras.layers.Dense(1000, activation='sigmoid')(input_0)
    # > layer for cdr3b alone
    input_1 = keras.layers.Input(shape=(len(cols_cdr3b)))
    output_1 = keras.layers.Dense(1000, activation='sigmoid')(input_1)
    # > layer for ag alone
    input_2 = keras.layers.Input(shape=(len(cols_ag)))
    output_2 = keras.layers.Dense(500, activation='sigmoid')(input_2)
    # > combined layer for tcra and tcrb
    concat_3 = keras.layers.Concatenate()([output_0, output_1])
    output_3 = keras.layers.Dense(1000, activation='sigmoid')(concat_3)
    # > combined layer for tcr and ag
    concat_4 = keras.layers.Concatenate()([output_2, output_3])
    output_4 = keras.layers.Dense(500, activation='sigmoid')(concat_4)
    # > final logit softmax layer
    output_5 = keras.layers.Dense(1, activation='sigmoid')(output_4)
    model = keras.Model(inputs=[input_0, input_1, input_2], outputs=[output_5])
    # set up the training parameters for the model
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss='binary_crossentropy',
        metrics=['accuracy','AUC'],
    )
    # train the model
    history = model.fit([X_train[cols_cdr3a], X_train[cols_cdr3b], X_train[cols_ag]], y_train,
                        epochs=10,
                        validation_data=([X_train[cols_cdr3a], X_train[cols_cdr3b], X_train[cols_ag]], y_train),
                        workers=40, use_multiprocessing=True)

    # testing predictions
    y_pred = model.predict([X_test[cols_cdr3a], X_test[cols_cdr3b], X_test[cols_ag]],
                           workers=40, use_multiprocessing=True)[:, 0]
    y_pred = pd.Series(y_pred, index=test.index, name='Pred%')

    hla2pred[hla] = y_pred

In [ ]:
# add back in predictions
test = test_orig.copy()
test['Prediction'] = np.nan
for pred in hla2pred.values():
    test.loc[pred.index, 'Prediction'] = pred
# write down predictions
test.to_csv('../external/tcr-specificity-prediction-challenge/submission.csv', index=False)

In [ ]:
import scipy.stats as ss
# define inputs with scores from immrep
df = pd.DataFrame([['TCRb','a',.6731],['TCRb','b',.6801],['TCRb','c',.67686131],
                   ['TCRb','d',.6891],['TCRb','e',.6819],
                   ['TCRa+b','a',.7583],['TCRa+b','b',.7635],['TCRa+b','c',.7621],
                   ['TCRa+b','d',.7360],['TCRa+b','e',.7539],], columns=['input','round','score'])
# create plot, set specific seed for jitter reproducibility
np.random.seed(27)
fig, ax = plt.subplots(figsize=[2, 4])
ax.grid(False)
sns.stripplot(x='input', y='score', hue='round', data=df, s=10, jitter=0.25)
ax.legend('', frameon=False)
# plot the average lines of each
mean_b = df.loc[df['input'] == 'TCRb', 'score'].mean()
mean_ab = df.loc[df['input'] == 'TCRa+b', 'score'].mean()
xlim = ax.get_xlim()
ylim = ax.get_ylim()
ax.plot(xlim, [mean_b]*2, linestyle='--', color='k')
ax.plot(xlim, [mean_ab]*2, linestyle='--', color='k')
ax.set_xlim(*xlim)
ax.set_ylim(ylim[0], ylim[1]+0.02)
# derive the p-value
p = ss.mannwhitneyu(df.loc[df['input'] == 'TCRb', 'score'], df.loc[df['input'] == 'TCRa+b', 'score'])[1]
print(p)

## fully upgraded model

In [ ]:
## DEFINE CONSTANTS
stretch_length_cdr3 = 30
stretch_length_ag = 15
aa_alphabet = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']
bcps = ['charge','hydrophobicity','weight','sulfur','aromatic']

## DEFINE ENCODERS
def _encode_peptide(peptide):
    # define columns
    idxs = range(len(peptide))
    cols = aa_alphabet+bcps
    # create the tracking dataframe
    X = pd.DataFrame(np.nan, index=idxs, columns=cols)
    for idx, aa in enumerate(peptide):
        # map to each of the bio-chem-physical components
        charge = amino_acid_charge[aa]
        hydrophobicity = amino_acid_hydrophobicity[aa] / 2
        weight = amino_acid_weights[aa] / stretch_length_cdr3
        sulfur = 1 * (aa in has_sulfur)
        aromatic = 1 * (aa in is_aromatic)
        # store in the final dataframe
        X.loc[idx, aa_alphabet] = 0
        X.loc[idx, aa] = 1
        X.loc[idx, bcps] = charge, hydrophobicity, weight, sulfur, aromatic
    assert not X.isna().any().any()
    return {peptide:X}
def encode_peptide(peptides, n_cpus):
    # work through peptides
    peptide_to_X = {}
    with mp.Pool(n_cpus) as pool:
        for result in pool.imap_unordered(_encode_peptide, peptides):
            peptide_to_X.update(result)
    return peptide_to_X

# retrieve encodings
cdr3a_to_X2 = encode_peptide(pd.concat([train_orig, test_orig], axis=0)['CDR3a_extended'].dropna().unique(), 40)
cdr3b_to_X2 = encode_peptide(pd.concat([train_orig, test_orig], axis=0)['CDR3b_extended'].dropna().unique(), 40)
ag_to_X2 = encode_peptide(pd.concat([train_orig, test_orig], axis=0)['Peptide'].dropna().unique(), 40)

In [ ]:
# retrieve hla specific training and testing data
hla2data2 = {}
for hla in test_orig['HLA'].unique():
    # subset for HLA
    train = train_orig.loc[train_orig['HLA'] == hla].copy()
    test = test_orig.loc[test_orig['HLA'] == hla].copy()
    train = train.dropna(subset=['CDR3a_extended'])

    from tqdm import tqdm
    # set seed and identify irrelevant matches
    np.random.seed(0)
    irrs = []

    # re count the peptides for this HLA
    counts = train['Peptide'].value_counts()
    for pep in tqdm(counts.index):
        # gather peptide cdr3 information
        n_cdr3s = counts.loc[pep]
        pep_cdr3s = train.loc[train['Peptide'] == pep, ['CDR3a_extended','CDR3b_extended']].value_counts().reset_index()
        # we first grab the peptide levenshtein distances
        pep_levenshtein = train_l[pep].sort_values()[::-1]
        # systematically identify negative controls
        irr_cdr3s = []
        while n_cdr3s > 0:
            # then we look at the max distance, gather those peptides, randomly choose one
            # find associated CDR3s that don't overlap with the current peptide
            vmax = pep_levenshtein.max()
            irr_peps = pep_levenshtein.index[pep_levenshtein == vmax]
            # reset for the next round if needed
            pep_levenshtein = pep_levenshtein[pep_levenshtein < vmax]
            # find the irrelevant peptide CDR3s and make sure they don't overlap
            mask = (train_orig['Peptide'].isin(irr_peps)) &\
            ((~train_orig['CDR3a_extended'].isin(pep_cdr3s['CDR3a_extended'])) |\
            (~train_orig['CDR3b_extended'].isin(pep_cdr3s['CDR3b_extended']))) &\
            (~train_orig['CDR3a_extended'].isna()) &\
            (~train_orig['CDR3b_extended'].isna())
            if sum(mask) == 0:
                continue
            # if we have cdr3s then grab them out
            cdr3s = train_orig.loc[mask, ['CDR3a_extended','CDR3b_extended']]
            # if there are more than we need randomly select
            if len(cdr3s) > n_cdr3s:
                irr_cdr3s_idxs = np.random.choice(cdr3s.index, size=n_cdr3s, replace=False)
                irr_cdr3s += cdr3s.loc[irr_cdr3s_idxs].values.tolist()
                break
            # otherwise keep moving
            else:
                irr_cdr3s += cdr3s.values.tolist()
                n_cdr3s -= len(cdr3s)
        # compile the full list
        irr = pd.DataFrame(irr_cdr3s, columns=['CDR3a_extended','CDR3b_extended'])
        irr['Peptide'] = pep
        irrs.append(irr)

    from sklearn.metrics import roc_curve, auc, accuracy_score

    ## CONVERT TO CORRECT FORMAT
    # create X for training
    X_train = pd.concat([pd.concat(irrs, axis=0), train], axis=0).reset_index().iloc[:, 1:]
    X_train_cdr3as, X_train_cdr3bs, X_train_ags = [], [], []
    for idx in X_train.index:
        # retrieve the data
        cdr3a, cdr3b, ag = X_train.loc[idx, 'CDR3a_extended'], X_train.loc[idx, 'CDR3b_extended'], X_train.loc[idx, 'Peptide']
        # grab the dataframes associated with the data
        X_tra, X_trb, X_ag = cdr3a_to_X2[cdr3a], cdr3b_to_X2[cdr3b], ag_to_X2[ag]
        # interpolate out to n=30 (no padding)
        X_tra2 = pd.DataFrame(index=range(stretch_length_cdr3), columns=X_tra.columns)
        for col in X_tra2.columns:
            X_tra2[col] = np.interp(range(stretch_length_cdr3), range(X_tra.shape[0]), X_tra[col])
        X_trb2 = pd.DataFrame(index=range(stretch_length_cdr3), columns=X_trb.columns)
        for col in X_trb2.columns:
            X_trb2[col] = np.interp(range(stretch_length_cdr3), range(X_trb.shape[0]), X_trb[col])
        # interpolate out to n=15 (no padding)
        X_ag2 = pd.DataFrame(index=range(stretch_length_ag), columns=X_ag.columns)
        for col in X_tra2.columns:
            X_ag2[col] = np.interp(range(stretch_length_ag), range(X_ag.shape[0]), X_ag[col])
        # append
        X_train_cdr3as.append(X_tra2)
        X_train_cdr3bs.append(X_trb2)
        X_train_ags.append(X_ag2)
    # grab y for training
    y_train = pd.Series(0, index=X_train.index)
    y_train.iloc[-train.shape[0]:] = 1
    # confirm the same length
    assert X_train.shape[0] == y_train.shape[0]

    # create X for testing
    X_test = test.copy()
    X_test_cdr3as, X_test_cdr3bs, X_test_ags = [], [], []
    for idx in X_test.index:
        # retrieve the data
        cdr3a, cdr3b, ag = X_test.loc[idx, 'CDR3a_extended'], X_test.loc[idx, 'CDR3b_extended'], X_test.loc[idx, 'Peptide']
        # grab the dataframes associated with the data
        X_tra, X_trb, X_ag = cdr3a_to_X2[cdr3a], cdr3b_to_X2[cdr3b], ag_to_X2[ag]
        # interpolate out to n=30 (no padding)
        X_tra2 = pd.DataFrame(index=range(stretch_length_cdr3), columns=X_tra.columns)
        for col in X_tra2.columns:
            X_tra2[col] = np.interp(range(stretch_length_cdr3), range(X_tra.shape[0]), X_tra[col])
        X_trb2 = pd.DataFrame(index=range(stretch_length_cdr3), columns=X_trb.columns)
        for col in X_trb2.columns:
            X_trb2[col] = np.interp(range(stretch_length_cdr3), range(X_trb.shape[0]), X_trb[col])
        # interpolate out to n=15 (no padding)
        X_ag2 = pd.DataFrame(index=range(stretch_length_ag), columns=X_ag.columns)
        for col in X_tra2.columns:
            X_ag2[col] = np.interp(range(stretch_length_ag), range(X_ag.shape[0]), X_ag[col])
        # append
        X_test_cdr3as.append(X_tra2)
        X_test_cdr3bs.append(X_trb2)
        X_test_ags.append(X_ag2)

    # transform all into numpy arrays
    X_train_cdr3as, X_train_cdr3bs, X_train_ags = np.array(X_train_cdr3as), np.array(X_train_cdr3bs), np.array(X_train_ags)
    X_test_cdr3as, X_test_cdr3bs, X_test_ags = np.array(X_test_cdr3as), np.array(X_test_cdr3bs), np.array(X_test_ags)
    hla2data2[hla] = X_train_cdr3as, X_train_cdr3bs, X_train_ags, X_test_cdr3as, X_test_cdr3bs, X_test_ags, y_train, test

In [ ]:
# retrieve hla specific predictions
hla2pred = {}
for hla in test_orig['HLA'].unique():
    # set seed
    np.random.seed(0)
    # read in all into numpy arrays
    X_train_cdr3as, X_train_cdr3bs, X_train_ags, X_test_cdr3as, X_test_cdr3bs, X_test_ags, y_train, test = hla2data2[hla]

    ## SETUP MODEL
    # > layer for tra
    input_1 = keras.layers.Input(shape=(stretch_length_cdr3, len(aa_alphabet)+len(bcps)))
    output_1 = keras.layers.Conv1D(30, 3, activation='gelu')(input_1)
    flatten_1a = keras.layers.Flatten()(output_1)
    flatten_1b = keras.layers.Flatten()(input_1)
    concat_1 = keras.layers.Concatenate()([flatten_1a, flatten_1b])
    output_1 = keras.layers.Dense(300, activation='sigmoid')(concat_1)
    # > layer for trb
    input_2 = keras.layers.Input(shape=(stretch_length_cdr3, len(aa_alphabet)+len(bcps)))
    output_2 = keras.layers.Conv1D(30, 3, activation='gelu')(input_2)
    flatten_2a = keras.layers.Flatten()(output_2)
    flatten_2b = keras.layers.Flatten()(input_2)
    concat_2 = keras.layers.Concatenate()([flatten_2a, flatten_2b])
    output_2 = keras.layers.Dense(300, activation='sigmoid')(concat_2)
    # > layer for ag
    input_3 = keras.layers.Input(shape=(stretch_length_ag, len(aa_alphabet)+len(bcps)))
    output_3 = keras.layers.Conv1D(15, 3, activation='gelu')(input_2)
    flatten_3a = keras.layers.Flatten()(output_3)
    flatten_3b = keras.layers.Flatten()(input_3)
    concat_3 = keras.layers.Concatenate()([flatten_3a, flatten_3b])
    output_3 = keras.layers.Dense(150, activation='sigmoid')(concat_3)
    # > concat tra + ag
    concat_13 = keras.layers.Concatenate()([output_1, output_3])
    output_13 = keras.layers.Dense(100, activation='sigmoid')(concat_13)
    # > concat trb + ag
    concat_23 = keras.layers.Concatenate()([output_2, output_3])
    output_23 = keras.layers.Dense(100, activation='sigmoid')(concat_23)
    # > concat tra + trb
    concat_12 = keras.layers.Concatenate()([output_1, output_2])
    output_12 = keras.layers.Dense(100, activation='sigmoid')(concat_12)
    # > concat tra+trb + ag
    concat_123 = keras.layers.Concatenate()([output_12, output_3])
    output_123 = keras.layers.Dense(100, activation='sigmoid')(concat_123)
    # > full combination
    concat_full = keras.layers.Concatenate()([output_12, output_13, output_123])
    output_full = keras.layers.Dense(1, activation='sigmoid')(concat_full)
    model = keras.Model(inputs=[input_1, input_2, input_3], outputs=[output_full])

    # set up the training parameters for the model
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss='binary_crossentropy',
        metrics=['accuracy','AUC'],
    )
    # train the model
    history = model.fit([X_train_cdr3as, X_train_cdr3bs, X_train_ags], y_train,
                        epochs=10,
                        validation_data=([X_train_cdr3as, X_train_cdr3bs, X_train_ags], y_train),
                        workers=40, use_multiprocessing=True)

    # testing predictions
    y_pred = model.predict([X_test_cdr3as, X_test_cdr3bs, X_test_ags],
                           workers=40, use_multiprocessing=True)[:, 0]
    y_pred = pd.Series(y_pred, index=test.index, name='Pred%')
    hla2pred[hla] = y_pred

# add back in predictions
test = test_orig.copy()
test['Prediction'] = np.nan
for pred in hla2pred.values():
    test.loc[pred.index, 'Prediction'] = pred
# write down predictions
test.to_csv('/ssd1/dchen/TMP/submission.csv', index=False)